In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import mutual_info_classif
import unicodedata
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from optuna.samplers import TPESampler
import optuna
import re
from fuzzywuzzy import process
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance
from catboost import Pool
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

c:\Users\Pc\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\Pc\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df = pd.read_csv(r'C:\Users\Pc\Desktop\twitter_multi_class_sentiment.csv')

In [4]:
df

,text,label,label_name
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger
...,...,...,...
15995,i just had a very brief time in the beanbag an...,0,sadness
15996,i am now turning and i feel pathetic that i am...,0,sadness
15997,i feel strong and good overall,1,joy
15998,i feel like this was such a rude comment and i...,3,anger


## 1. Tokenizasyon ve Veri Setini Hazırlama

In [5]:
from transformers import AutoTokenizer

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [6]:
text = "I love machine learning! Tokenization is awesome!!"
encoded_text = tokenizer(text)
print(encoded_text)

{'input_ids': [101, 1045, 2293, 3698, 4083, 999, 19204, 3989, 2003, 12476, 999, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
tokenizer.vocab_size, tokenizer.model_max_length

(30522, 512)

## 2. DataSet, Train-Test-Validation Split

In [8]:
train, test = train_test_split(df, test_size=0.3, stratify=df['label_name'])
test, validation = train_test_split(test, test_size=1/3, stratify=test['label_name'])

train.shape, test.shape, validation.shape

((11200, 3), (3200, 3), (1600, 3))

In [9]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict(
    {'train':Dataset.from_pandas(train, preserve_index=False),       # indeksleri almıyor çünkü indeksler rastgele alındığı için farklı
     'test':Dataset.from_pandas(test, preserve_index=False),
     'validation': Dataset.from_pandas(validation, preserve_index=False)
     }  
)

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name'],
        num_rows: 11200
    })
    test: Dataset({
        features: ['text', 'label', 'label_name'],
        num_rows: 3200
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name'],
        num_rows: 1600
    })
})

In [10]:
dataset['train'][0]

{'text': 'i feel a bit frightened that you are touching my car while i am away repeatedly i ask you to stop putting stuff on it',
 'label': 4,
 'label_name': 'fear'}

## 3. Tokenization of the Emotion/Sentiment Data

In [11]:
def tokenize(df):
    temp = tokenizer(df['text'], padding=True, truncation=True)  # Padding 0 koyuyor yeteri uzunlukta değilse, truncation uzun olanı kesiyor cümlenin 
    return temp

print(tokenize(dataset['train'][:2]))

{'input_ids': [[101, 1045, 2514, 1037, 2978, 10363, 2008, 2017, 2024, 7244, 2026, 2482, 2096, 1045, 2572, 2185, 8385, 1045, 3198, 2017, 2000, 2644, 5128, 4933, 2006, 2009, 102, 0, 0, 0, 0], [101, 10047, 2074, 3110, 7653, 2000, 2031, 3856, 2039, 2256, 2879, 2077, 2027, 3844, 1996, 2173, 2091, 1998, 8243, 1996, 6077, 1998, 3407, 2000, 2031, 5407, 2032, 2041, 1997, 2045, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [12]:
dataset['train'][:2]

{'text': ['i feel a bit frightened that you are touching my car while i am away repeatedly i ask you to stop putting stuff on it',
  'im just feeling relieved to have picked up our boy before they shut the place down and seized the dogs and happy to have gotten him out of there'],
 'label': [4, 1],
 'label_name': ['fear', 'joy']}

In [13]:
emotion_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/11200 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [14]:
# label2id, id2label
label2id = {x['label_name']:x['label'] for x in dataset['train']}
id2label = {v:k for k,v in label2id.items()}

label2id, id2label

({'fear': 4, 'joy': 1, 'sadness': 0, 'anger': 3, 'surprise': 5, 'love': 2},
 {4: 'fear', 1: 'joy', 0: 'sadness', 3: 'anger', 5: 'surprise', 2: 'love'})

## 4. Model Building

In [15]:
from transformers import AutoModel 
import torch

In [16]:
model = AutoModel.from_pretrained(model_ckpt)

In [17]:
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

## 5. Fine-Tuning Transformers

- `AutoModelForSequenceClassification` model has a classification head on top of the pretrained model outputs

- The first thing we need is a pretrained BERT model like the one we used in the feature-based approach.
- The only slight modification is that we use the `AutoModelForSequenceClassification` model instead of AutoModel.
- The difference is that the `AutoModelForSequenceClassification` model has a classification head on top of the pretrained model outputs, which can be easily trained with the base model.

In [18]:
from transformers import AutoModelForSequenceClassification, AutoConfig

num_labels = len(label2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "sadness",
    "1": "joy",
    "2": "love",
    "3": "anger",
    "4": "fear",
    "5": "surprise"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "fear": 4,
    "joy": 1,
    "love": 2,
    "sadness": 0,
    "surprise": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [55]:
from transformers import TrainingArguments

batch_size = 64
training_dir = r"C:\Temp\bert_base_train_dir"

training_args = TrainingArguments(output_dir=training_dir,
                                  overwrite_output_dir = True,
                                  num_train_epochs = 2,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = 'epoch',
                                  disable_tqdm = False
)

## 6. Build Model and Trainer

In [56]:
# use sklearn to build compute metrics
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "f1": f1}


In [57]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset = emotion_encoded['train'],
                  eval_dataset = emotion_encoded['validation'],
                  tokenizer = tokenizer)

In [58]:
trainer.train()

  0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.5087429881095886, 'eval_accuracy': 0.84625, 'eval_f1': 0.8398980265291334, 'eval_runtime': 90.498, 'eval_samples_per_second': 17.68, 'eval_steps_per_second': 0.276, 'epoch': 1.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.3060774505138397, 'eval_accuracy': 0.89875, 'eval_f1': 0.8982473568514764, 'eval_runtime': 180.5574, 'eval_samples_per_second': 8.861, 'eval_steps_per_second': 0.138, 'epoch': 2.0}
{'train_runtime': 5012.036, 'train_samples_per_second': 4.469, 'train_steps_per_second': 0.07, 'train_loss': 0.7386039515904018, 'epoch': 2.0}


TrainOutput(global_step=350, training_loss=0.7386039515904018, metrics={'train_runtime': 5012.036, 'train_samples_per_second': 4.469, 'train_steps_per_second': 0.07, 'total_flos': 1001502421516800.0, 'train_loss': 0.7386039515904018, 'epoch': 2.0})

## 7. MODEL EVALUATION

In [59]:
preds_output = trainer.predict(emotion_encoded['test'])
preds_output.metrics

  0%|          | 0/50 [00:00<?, ?it/s]

{'test_loss': 0.30711495876312256,
 'test_accuracy': 0.9084375,
 'test_f1': 0.9087661769652071,
 'test_runtime': 214.576,
 'test_samples_per_second': 14.913,
 'test_steps_per_second': 0.233}

In [60]:
y_pred = np.argmax(preds_output.predictions, axis=1)
y_true = emotion_encoded['test'][:]['label']

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       933
           1       0.94      0.91      0.93      1072
           2       0.79      0.84      0.81       261
           3       0.91      0.92      0.91       432
           4       0.86      0.88      0.87       387
           5       0.83      0.82      0.82       115

    accuracy                           0.91      3200
   macro avg       0.88      0.89      0.88      3200
weighted avg       0.91      0.91      0.91      3200

